<a href="https://colab.research.google.com/github/vadlasushma/Sentiment_Analysis_legal/blob/main/Retrieve_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
#get object id's of documents in the docket
def get_doc_id(DEMO_KEY,docketId) :
  response_API = requests.get('https://api.regulations.gov/v4/documents?filter[docketId]='+docketId+'&api_key=DEMO_KEY')
  #print(response_API.status_code)
  comment_single = response_API.text
  parse_comments = json.loads(comment_single)
  print("The list of documents in docket",comment_single )
  #get the objectID of documents
  objIds=[]
  for i in range(len(parse_comments)) :
    objIds.append(parse_comments['data'][i]['attributes']['objectId'])
  print('ObjectIDs',objIds)
  return objIds

In [71]:
#links for comments of document 1
def get_comment_links(DEMO_KEY,objIds,links) :
  page_size = 250
  comment_onID = objIds
  for i in range(1,20):
    path = 'https://api.regulations.gov/v4/comments?filter[commentOnId]={}&page[size]={}&page[number]={}&sort=lastModifiedDate,documentId&api_key={}'.format (comment_onID, page_size, i, DEMO_KEY )
    response_API = requests.get(path)
    all_comments = response_API.text
    parse_comments = json.loads(all_comments)
    x = response_API.headers["X-RateLimit-Remaining"]
    if (x!='0') :
      print("Retrieved links for page {} is {}".format(i,len(parse_comments['data'])))
      for j in range(len(parse_comments['data'])-1) :
        try:
          links.append(parse_comments['data'][j]['links']['self'])
        except:
          print (j)
    else : 
      return 0
            
  return links

In [72]:
#each comment data
def get_comment_texts(DEMO_KEY,links) :
  str1 = '?api_key='+DEMO_KEY
  texts = []
  count = 0
  for i in links :
    temp = []
    count=count+1
    str2=i+str1
    response_API = requests.get(str2)
    comment_single = response_API.text
    comments_parse = json.loads(comment_single)
    x = response_API.headers["X-RateLimit-Remaining"]
    if (x!='0') :
      print(count)
      temp.append(comments_parse['data']['attributes']['docketId'])
      temp.append(comments_parse['data']['attributes']['documentType'])
      temp.append(comments_parse['data']['attributes']['govAgency'])
      temp.append(comments_parse['data']['attributes']['firstName'])
      temp.append(comments_parse['data']['attributes']['lastName'])
      temp.append(comments_parse['data']['attributes']['organization'])
      temp.append(comments_parse['data']['attributes']['comment'])
      texts.append(temp)
    else :
      print('Limit exceeded')
      print('# of comments retrieved is', count-1)
      break
  return texts

In [ ]:
#main
import requests
import json
import pandas as pd
DEMO_KEY="EzpoTEvbrW48r9dT952ItW0SGwptBe8BR0yBKlIe"
# DEMO_KEY = "qFXZRLl65B5F4w8b5potNh6llprE7YEQZkncFn2x"
docketId = 'ATF-2021-0001'
#obj id's of document
objIds = get_doc_id(DEMO_KEY,docketId)
links = []
#get comment links using each obj id of docs
for i in range(len(objIds)) :
  temp = []
  a = get_comment_links(DEMO_KEY,objIds[i],temp)
  if(a==0) :
    print('Limit exceeded')
  else :
    links.append(a)  
all_text = []  
#get comment texts using comment links
for i in range(len(links)) :  
  all_text.append(get_comment_texts(DEMO_KEY,links[i]))
# write the text to file  
for i in range(len(all_text)) :
  df1=pd.DataFrame(all_text[i], columns = ['docketId','documentType','govAgency','firstName','lastName','organization','Comment'])
  with open(r'./retrieved_comments'+str(i)+'.csv', 'w') as file:
    print('Writing to file')
    df1.to_csv(file)



